# Обучение модели классификации комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель исследования** - обучить модель, которая классифицирует комментарии на позитивные и негативные.

**В рамках исследования необходимо:**

 - Подготовить и обработать данные 
 - Лемматизировать текст и оичистить от лишних сииволов
 - Токенизировать текст
 - Проверить баланс классов в целевом признаке
 - Разбить данные на выборки (тренировочную и тестовую)
 - Выделить в выборках признаки и целевой признак
 - Обучить модели (LinearRegression, CatBoostRegressor, RandomForestClassifier) 
 - Проверить лучшую модель на тестовой выборке
 - Значеие метрики F1 должно быть меньше 0.75

Сдеалть вывод с рекомендацией наилучшей модели заказчику.

**В нашем распоряжении набор данных с разметкой о токсичности правок.**

## Подготовка

In [1]:
!pip install imbalanced-learn

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/elenarezvakova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/elenarezvakova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:

import pandas as pd
import numpy as np 

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk import pos_tag

from sklearn.feature_extraction.text import TfidfVectorizer 
import re 

import matplotlib.pyplot as plt

from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.pipeline import Pipeline

import catboost as cb
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

In [5]:
data = pd.read_csv('toxic_comments.csv')

In [6]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Итак, пропусков в данных нет, имеется столбец Unnamed который дублирует порядковый номер, удалим его

In [8]:
data = data.drop('Unnamed: 0', axis=1)

In [9]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [10]:
data.duplicated().sum()

0

Видим лишние символы в тексте, **очистим текст** от них

In [11]:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [12]:
%%time
#очищаю тексты постов:
data['text'] = data['text'].apply(clear_text) 

CPU times: user 3.37 s, sys: 28.7 ms, total: 3.4 s
Wall time: 3.4 s


**Лематизируем** текст


In [13]:
%%time

#ввожу функцию РОS-тэгирования слов:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,              #существительное
                "V": wordnet.VERB,              #глагол
                "R": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

#ввожу функцию лемматизации тектов:
def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


In [14]:
from tqdm.notebook import tqdm
tqdm.pandas()

data['lemm_text'] = data['text'].progress_apply(lemm_text) 


  0%|          | 0/159292 [00:00<?, ?it/s]

In [15]:
data.head()

,text,toxic,lemm_text
0,explanation why the edits made under my userna...,0,explanation why the edits make under my userna...
1,d aww he matches this background colour i m se...,0,d aww he match this background colour i m seem...
2,hey man i m really not trying to edit war it s...,0,hey man i m really not try to edit war it s ju...
3,more i can t make any real suggestions on impr...,0,more i can t make any real suggestion on impro...
4,you sir are my hero any chance you remember wh...,0,you sir be my hero any chance you remember wha...


Обработку провели, текст почистили, лематизировали теперь посмотри на **соотношение классов** в целевом признаке

In [16]:
data['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Итак, **видим дисбаланс** классов соотношение примерно 90% на 10 %. Будем проводить балансировку классов методом **RandomUnderSampler**, таким образом уменьшим выборку мажоритарного класса.

Создадим переменные для **признаков** и **целевого признака**

In [17]:
features = data['lemm_text']
target = data['toxic']

Так как в нашем распоряжении одна база, то нам необходимо разбить ее на две части: **обучающую и тестовую**. Выделим части в соотношении 90%-10% соответственно.

In [18]:
features_train, features_test, target_train, target_test = (
    train_test_split(features, target, test_size=0.1, random_state=12345))

In [19]:
#Проверим размеры выборок
len(features_train)

143362

In [20]:
len(features_test)

15930

создадм матрицу cо значениями **TF-IDF** по корпусу

In [21]:
corpus_train = features_train.values
corpus_test = features_test.values

In [22]:

stopwords = list(set(nltk_stopwords.words('english')))

In [23]:
stopwords

['couldn',
 "should've",
 'any',
 's',
 'they',
 'other',
 'further',
 "won't",
 'each',
 'while',
 'can',
 "couldn't",
 't',
 'he',
 'was',
 'these',
 'from',
 'against',
 'up',
 "hasn't",
 'than',
 'mustn',
 'over',
 'have',
 'hadn',
 'as',
 'did',
 'most',
 'his',
 'ain',
 "shan't",
 'yourselves',
 'their',
 'by',
 'we',
 'if',
 'not',
 "that'll",
 'him',
 "she's",
 'to',
 'me',
 'should',
 'my',
 'both',
 'will',
 'and',
 'does',
 'won',
 'hasn',
 'under',
 'do',
 'how',
 'been',
 'having',
 'your',
 'for',
 "don't",
 'between',
 'needn',
 "didn't",
 'hers',
 'you',
 'where',
 'shan',
 'herself',
 "mustn't",
 'on',
 "shouldn't",
 'own',
 'very',
 "aren't",
 'some',
 "isn't",
 "you'll",
 'in',
 'those',
 'himself',
 'is',
 'once',
 'a',
 'here',
 'until',
 'but',
 'into',
 'below',
 'too',
 "wasn't",
 'themselves',
 'before',
 'y',
 'doing',
 'weren',
 'whom',
 'that',
 'such',
 'above',
 "mightn't",
 'what',
 'them',
 "needn't",
 'she',
 'off',
 'didn',
 'so',
 'all',
 'her',
 'll'

In [24]:
%%time
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train =  count_tf_idf.fit_transform(corpus_train)
tf_idf_test =  count_tf_idf.transform(corpus_test)

CPU times: user 5.02 s, sys: 69.8 ms, total: 5.09 s
Wall time: 5.09 s


In [25]:
tf_idf_train.shape

(143362, 142204)

In [26]:
tf_idf_test.shape

(15930, 142204)

In [27]:
target_test.shape

(15930,)

**Вывод по этапу подготовка данных**: был удален ненужный столбец, тексты были очищены от лишних знаков, лематизированы, была проведена балансирровка классов по целевому признаку, созданы матрицы признаков для обуччающей и тестовой выборки.  Можно переходить к обучению моделей.

## Обучение

**LogisticRegression**

In [29]:

from sklearn.metrics import make_scorer, f1_score
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)), 
    ('clf', LogisticRegression())
])

parameters = {'clf__C': [5,10,15]} # список значений C, которые нужно перебрать

grid_search = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1, scoring='f1')
grid_search.fit(features_train, target_train)

model = grid_search.best_estimator_
print("Best C parameter: ", grid_search.best_params_['clf__C'])
print("Best F1 score: ", grid_search.best_score_)



/Users/elenarezvakova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/elenarezvakova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linea

Best C parameter:  10
Best F1 score:  0.7671333048494579


/Users/elenarezvakova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**LogisticRegression с балансировкой**

In [30]:
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE

steps = [
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('smote', SMOTE()),
    ('logreg', LogisticRegression(C=10))
]
pipeline_smote = Pipeline(steps=steps)
scores = cross_val_score(pipeline_smote,features_train, target_train, cv=3, scoring='f1')
print('Average cross-validation score:', scores.mean())



/Users/elenarezvakova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/elenarezvakova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linea

Average cross-validation score: 0.6797274592841305


**CatBoostClassifier без учета баланса классов**

In [31]:



pipeline_cb = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('classifier', CatBoostClassifier()
])

scores_cb = cross_val_score(pipeline_cb,features_train, target_train, cv=3, scoring='f1')
print('Average cross-validation score:', scores_cb.mean())


Learning rate set to 0.0722
0:	learn: 0.6208289	total: 579ms	remaining: 9m 38s
1:	learn: 0.5602183	total: 986ms	remaining: 8m 11s
2:	learn: 0.5066763	total: 1.32s	remaining: 7m 20s
3:	learn: 0.4637010	total: 1.72s	remaining: 7m 8s
4:	learn: 0.4259483	total: 2.07s	remaining: 6m 51s
5:	learn: 0.3949476	total: 2.4s	remaining: 6m 37s
6:	learn: 0.3694466	total: 2.72s	remaining: 6m 25s
7:	learn: 0.3481890	total: 3.04s	remaining: 6m 16s
8:	learn: 0.3289822	total: 3.36s	remaining: 6m 9s
9:	learn: 0.3132393	total: 3.68s	remaining: 6m 4s
10:	learn: 0.3004981	total: 4s	remaining: 5m 59s
11:	learn: 0.2896586	total: 4.32s	remaining: 5m 55s
12:	learn: 0.2790873	total: 4.66s	remaining: 5m 53s
13:	learn: 0.2710155	total: 5s	remaining: 5m 52s
14:	learn: 0.2641447	total: 5.33s	remaining: 5m 49s
15:	learn: 0.2580395	total: 5.65s	remaining: 5m 47s
16:	learn: 0.2528481	total: 5.98s	remaining: 5m 45s
17:	learn: 0.2483973	total: 6.3s	remaining: 5m 43s
18:	learn: 0.2444482	total: 6.62s	remaining: 5m 41s
19:	l

**RandomForestClassifier с балансировкой SMOTE** 

In [32]:
%%time
from imblearn.pipeline import Pipeline
from sklearn.metrics import f1_score

# Создаем Pipeline
pipeline_rfs = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('smote', SMOTE()),
    ('clf', RandomForestClassifier())
])

# Определяем диапазон гиперпараметров для GridSearchCV
parameters_rfs = {
    'clf__n_estimators': [50, 80],
    'clf__max_depth': [5, 7, 10]
}

# Запускаем GridSearchCV с метрикой f1
grid_search_rfs = GridSearchCV(pipeline_rfs, parameters_rfs, cv=3, scoring='f1')

# Обучаем модель
grid_search_rfs.fit(features_train, target_train)

# Выводим лучшую модель
print("Best parameters: {}".format(grid_search_rfs.best_params_))
print("Best F1 score: {:.2f}".format(grid_search_rfs.best_score_))



Best parameters: {'clf__max_depth': 10, 'clf__n_estimators': 80}
Best F1 score: 0.37
CPU times: user 11min, sys: 9.44 s, total: 11min 9s
Wall time: 4min 22s


Таким образом, лучше всего себя показала модель LogisticRegression с параметром С=10, f1-мера даннной модели равна 0,78. Хуже всего проявила себя модель RandomForestClassifier.

**Тестирование модели лучшей модели**

In [33]:
predictions_test = model.predict(features_test)
lr_test_f1 = f1_score(target_test, predictions_test)
print("f1 на тестовых данных:",lr_test_f1)

f1 на тестовых данных: 0.7778551532033426


## Выводы

Необходимо было построить модель для классификации комментариев на позитивные и негативные и учесть, чтобы значение метрики F1 на тестовой выборке было не меньше 0.75.
Мы использовали для обучения модели CatBoostRegressor, LinearRegression, RandomForestRegressor с учетом балансов классов в целевом признаке и без.

По итогам качества обучения побеждает модель ***LogisticRegression*** , рекомендуем данную модель для классификации комментариев в новом сервисе интернет-магазина.
